In [1]:
#%pip install pybamm -q    # install PyBaMM if it is not installed
import os
import matplotlib.pyplot as plt
import numpy as np
import pybamm
import pandas as pd
import timeit
from matplotlib import style
style.use('ggplot')
start = timeit.default_timer()
os.chdir(pybamm.__path__[0]+'/..')
pybamm.set_logging_level("INFO")

In [ ]:
model = pybamm.lithium_ion.BasicDFNComposite()
chemistry = pybamm.parameter_sets.Chen2020_composite
param = pybamm.ParameterValues(chemistry=chemistry)
# model = pybamm.lithium_ion.DFN()
# param = model.default_parameter_values
param.update({"Upper voltage cut-off [V]": 4.5})
param.update({"Lower voltage cut-off [V]": 2})
param.update({
    "Maximum concentration in negative electrode of phase 1 [mol.m-3]":28700,
    "Initial concentration in negative electrode of phase 1 [mol.m-3]":23000,
    "Negative electrode mass fraction of phase 1":0.98,
    "Negative electrode mass fraction of phase 2":0.02,
    "Initial concentration in negative electrode of phase 2 [mol.m-3]":277000,
    "Maximum concentration in negative electrode of phase 2 [mol.m-3]":278000,
    })

# initial concentration of graphite 29866 [mol.m-3]
# -> 0.9134 * 25830 + 0.0866 * 275220

C_rates = 0.01
Q_cell = 4.89 # [Ah]
period = 3600 / C_rates * Q_cell / 5
rest_period = 3600
rest_period2 = 3600
transition = period / 400
timestep=[rest_period, period, rest_period2, period]
points=[101,401,101,401]
t_evals=[]
t0 = 0
for i in range(0,len(timestep)):
    t1 = np.linspace(t0, t0 + timestep[i] - transition, points[i])
    t0 += timestep[i]
    if i<len(timestep)-1:
        t2 = np.linspace(t0 - transition, t0 + transition, 41) # transition points
    else:
        t2 = []
    t_evals=np.concatenate((t_evals, t1 ,t2))

t_evals = np.unique(t_evals)
capacity = param["Nominal cell capacity [A.h]"]
I_load = C_rates * capacity  

def I_fun(A):
    def current(t):
        C_rates = 0.01
        Q_cell = 4.89 # [Ah]
        period = 3600 / C_rates * Q_cell / 5
        rest_period = 3600
        rest_period2 = 3600
        k = 20
        rest = pybamm.sigmoid( t , rest_period, k)
        discharge = pybamm.sigmoid( t , rest_period + period, k) - rest
        # rest3 = pybamm.sigmoid( rest_period + period * 2 + rest_period2, t, k)
        charge = pybamm.sigmoid( rest_period + period + rest_period2, t, k)
        return A * discharge - A * charge
        # return A * (t < period) - A * (t >= period)
        # return A * 0.2 * pybamm.sin(2 * np.pi * t / 7200 )
    return current


param["Current function [A]"] = I_fun(I_load)

sim1 = pybamm.Simulation(
    model,
    parameter_values=param,
    solver=pybamm.CasadiSolver(dt_max = 10),
)
solution = sim1.solve(t_eval = t_evals)
stop = timeit.default_timer()
print("running time: " + str(stop - start) + "s")

2021-07-26 09:12:20,988 - [INFO] parameter_values.process_model(411): Start setting parameters for Doyle-Fuller-Newman model
2021-07-26 09:12:21,212 - [INFO] parameter_values.process_model(501): Finish setting parameters for Doyle-Fuller-Newman model
2021-07-26 09:12:21,214 - [INFO] discretisation.process_model(137): Start discretising Doyle-Fuller-Newman model
2021-07-26 09:12:22,322 - [INFO] discretisation.process_model(239): Finish discretising Doyle-Fuller-Newman model
2021-07-26 09:12:22,323 - [INFO] base_solver.solve(554): Start solving Doyle-Fuller-Newman model with CasADi solver with 'safe' mode
2021-07-26 09:12:22,328 - [INFO] base_solver.set_up(140): Start solver set-up
2021-07-26 09:12:22,706 - [INFO] base_solver.set_up(426): Finish solver set-up


In [ ]:
# plot
plot = pybamm.QuickPlot( solution, [       
        "Current [A]",
        "Terminal voltage [V]",
        "X-averaged negative electrode open circuit potential of phase 1 [V]",
        "X-averaged negative electrode open circuit potential of phase 2 [V]",
    ]
)
plot.dynamic_plot()


In [ ]:
current = solution["Current [A]"].entries
times = solution["Time [s]"].entries / 3600
voltage = solution["Terminal voltage [V]"].entries
ocp_p1 = solution["X-averaged negative electrode open circuit potential of phase 1 [V]"].entries
ocp_p2 = solution["X-averaged negative electrode open circuit potential of phase 2 [V]"].entries
ocp_chen = pd.read_csv ("wip/data/negative_ocp_LGM50.txt", delimiter= '\s+',header=None,comment='#')
capacity = current * 0
capacity_max = 0
time_discharge = 0
Q_t = 0
for i in range(1,len(current)):
    Q_t += (current[i] + current[i - 1]) / 2 * (times[i] - times[i-1])
    capacity[i] = Q_t
capacity = Q_cell - capacity

In [ ]:
V1_Niall = pd.read_csv ("wip/data/NDK01_14_001C_1.txt", delimiter= '\s+',header=1,comment='#')
V2_Niall = pd.read_csv ("wip/data/NDK01_14_001C_2.txt", delimiter= '\s+',header=1,comment='#')
plt.plot(capacity,voltage,'b--',label='Simulation')
plt.plot(Q_cell - V2_Niall.values[:,4]/1000,V2_Niall.values[:,0],'r-',label='Experiment')
plt.plot(V1_Niall.values[:,4]/1000,V1_Niall.values[:,0],'r-',label=None)
plt.xlabel("Capacity [Ah]")
plt.ylabel("Voltage [V]")
plt.title("Cell voltage [V]")
plt.legend()

In [ ]:
plt.plot(capacity,ocp_p1,'r--',label='Graphite')
plt.plot(capacity,ocp_p2,'b:',label='Silicon')
plt.plot(ocp_chen.values[:,0],ocp_chen.values[:,1],'k-',label='Composite (Chen et al. 2020)')
plt.xlabel("Capacity [Ah]")
plt.ylabel("Equilibrium potential [V]")
plt.title("C/100")
plt.legend()

In [ ]:
plt.plot(times,ocp_p1,'r--',label='Graphite')
plt.plot(times,ocp_p2,'b:',label='Silicon')
plt.xlabel("Time [h]")
plt.ylabel("Equilibrium potential [V]")
plt.title("C/100")
plt.legend()
# plt.xlim([100, 103])
# plt.ylim([0.4, 1])

In [ ]:
c_s_xrav_n_p1 = solution["Averaged negative electrode concentration of phase 1"].entries
c_s_xrav_n_p2 = solution["Averaged negative electrode concentration of phase 2"].entries
plt.plot(capacity, c_s_xrav_n_p1,'r-', label='Graphite')
plt.plot(capacity, c_s_xrav_n_p2, 'b:',label='Silicon')
plt.xlabel("Capacity [Ah]")
plt.ylabel("$c/c_\mathrm{max}$")
# plt.ylim([0,0.02])
plt.legend()

In [ ]:
plt.plot(c_s_xrav_n_p1, ocp_p1,'r-', label='Graphite')
plt.plot(c_s_xrav_n_p2, ocp_p2, 'b:',label='Silicon')
plt.xlabel("$c/c_\mathrm{max}$")
plt.ylabel("Equilibrium potential [V]")
plt.title("C/100")
# plt.ylim([0,0.02])
plt.legend()

In [ ]:
j_n_p1_av = solution["X-averaged negative electrode interfacial current density of phase 1 [A.m-2]"].entries
j_n_p2_av = solution["X-averaged negative electrode interfacial current density of phase 2 [A.m-2]"].entries
plt.plot(times, j_n_p1_av,'r-', label='Graphite')
plt.plot(times, j_n_p2_av, 'b:',label='Silicon')
plt.xlabel("Time [h]")
plt.ylabel("Interfacial current density [A/m$^{2}$]")
# plt.ylim([-0.2, 0.8])
# plt.xlim([96, 102])
plt.legend()

In [ ]:
j_n_p1_av = solution["X-averaged negative electrode interfacial current density of phase 1 [A.m-2]"].entries
j_n_p2_av = solution["X-averaged negative electrode interfacial current density of phase 2 [A.m-2]"].entries
plt.plot(times, j_n_p1_av,'r-', label='Graphite')
plt.plot(times, j_n_p2_av, 'b:',label='Silicon')
plt.xlabel("Time [h]")
plt.ylabel("Interfacial current density [A/m$^{2}$]")
# plt.ylim([-0.2, 0.8])
plt.xlim([96, 102])
plt.legend()

In [ ]:
return

In [ ]:
c_n_max= param["Maximum concentration in negative electrode [mol.m-3]"]
F = 96485.3329 #s A / mol
L_n =  param["Negative electrode thickness [m]"]
L_p =  param["Positive electrode thickness [m]"]
L_s =  param["Separator thickness [m]"]
I_typ = param["Typical current [A]"]
ne= param["Number of electrodes connected in parallel to make a cell"]
L_width = param["Electrode width [m]"]
L_height = param["Electrode height [m]"]
timescale = c_n_max * F * (L_p + L_n + L_s) / abs(I_typ / (ne * L_width * L_height))
print(timescale)

In [ ]:
print(min(c_s_xrav_n_p1))

In [ ]:
print(range(0,len(timestep)))